# Only Pz remains for many synaptic inputs

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
from lfpykit.models import CurrentDipoleMoment
from brainsignals.plotting_convention import mark_subplots
from brainsignals import neural_simulations as ns

In [ ]:
np.random.seed(1234)
dt = 2**-4
tstop = 20

# Define synapse parameters
synapse_params = {
    'e' : 0.,                   # reversal potential
    'syntype' : 'Exp2Syn',       # synapse type
    'tau1' : 0.1,                 # synaptic time constant
    'tau2' : 1.,                 # synaptic time constant
    'weight' : 0.001,            # synaptic weight
    'record_current' : False,    # record synapse current
}

cell = ns.return_hay_cell(tstop, dt, make_passive=True)

# Store for plotting
cell_x, cell_z = cell.x.copy(), cell.z.copy()
synidxs = cell.get_rand_idx_area_norm(z_min=-1e9, z_max=300, nidx=100)
del cell

In [ ]:
cdms = []
tvec = None
for s_, synidx in enumerate(synidxs):
    print(s_ + 1, "/", len(synidxs))
    cell = ns.return_hay_cell(tstop, dt, make_passive=True)

    synapse_params["idx"] = synidx
    synapse = LFPy.Synapse(cell, **synapse_params)
    synapse.set_spike_times(np.array([1.]))

    cell.simulate(rec_imem=True, rec_vmem=True)
    cdm = CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem
    cdms.append(cdm.copy())
    if s_ == 0:
        tvec = cell.tvec.copy()
    del cell
    del synapse
    del cdm

# Now doing simultaneous input
cell = ns.return_hay_cell(tstop, dt, make_passive=True)
for s_, synidx in enumerate(synidxs):
    synapse_params["idx"] = synidx
    synapse = LFPy.Synapse(cell, **synapse_params)
    synapse.set_spike_times(np.array([1.]))

cell.simulate(rec_imem=True, rec_vmem=True)
cdms_sum = CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem

cdm_max = 3 #np.max(np.abs(cdms)) * 0.75
cdm_sum_max = 60 #np.max(np.abs(cdms_sum)) * 0.75


In [ ]:

fig = plt.figure()
fig.subplots_adjust(bottom=0.01, top=0.9, right=0.99, left=0.23,
                    wspace=0.3, hspace=0.3,)
ax0 = fig.add_axes([0.0, 0.05, 0.15, 0.9], aspect=1,
                   frameon=False, xticks=[], yticks=[])

ax_dict = dict(ylim=[-cdm_max, cdm_max],
               frameon=False, xticks=[], yticks=[])
ax_ind_1 = fig.add_subplot(321, **ax_dict)
ax_ind_2 = fig.add_subplot(323, **ax_dict)
ax_ind_3 = fig.add_subplot(325, **ax_dict)

ax_dict = dict(ylim=[-cdm_sum_max, cdm_sum_max],
               frameon=False, xticks=[], yticks=[])
ax_sum_1 = fig.add_subplot(322, **ax_dict)
ax_sum_2 = fig.add_subplot(324, **ax_dict)
ax_sum_3 = fig.add_subplot(326, **ax_dict)

ax_ind_1.text(-3, 0, "$P_x$", va="center")
ax_ind_2.text(-3, 0, "$P_y$", va="center")
ax_ind_3.text(-3, 0, "$P_z$", va="center")

ax_sum_1.text(-3, 0, "$P_x$", va="center")
ax_sum_2.text(-3, 0, "$P_y$", va="center")
ax_sum_3.text(-3, 0, "$P_z$", va="center")

fig.text(0.35, 0.98, "individual inputs", ha="center", va="top")
fig.text(0.8, 0.98, "simultaneous inputs", ha="center", va="top")

ax0.plot(cell_x.T, cell_z.T, c='k')
ax0.plot(cell_x[synidxs].mean(axis=1), cell_z[synidxs].mean(axis=1), 'o',
         ms=2, c='b')

ax_ind_3.plot([tstop - 6, tstop - 6], [1, 1+cdm_max], clip_on=False, c='k', lw=1)
ax_ind_3.text(tstop - 5.7, 1 + cdm_max/2, "{:d} nA µm".format(int(cdm_max)),
              ha='left', va="center")

ax_sum_3.plot([tstop - 6, tstop - 6], [40, 40+cdm_sum_max], clip_on=False, c='k', lw=1)
ax_sum_3.text(tstop - 5.7, 40 + cdm_sum_max/2, "{:d} nA µm".format(int(cdm_sum_max)),
              ha='left', va="center")

ax_ind_1.plot([tstop - 5, tstop], [-1, -1], c='k', lw=1)
ax_ind_1.text(tstop - 2.5, -1.3, "5 ms", ha="center", va="top")

ax_sum_1.plot([tstop - 5, tstop], [-18, -18], c='k', lw=1)
ax_sum_1.text(tstop - 2.5, -25, "5 ms", ha="center", va="top")

for s_, synidx in enumerate(synidxs):
    ax_ind_1.plot(tvec, cdms[s_][0, :], c='k', clip_on=False, lw=0.3)
    ax_ind_2.plot(tvec, cdms[s_][1, :], c='k', clip_on=False, lw=0.3)
    ax_ind_3.plot(tvec, cdms[s_][2, :], c='k', clip_on=False, lw=0.3)

ax_sum_1.plot(tvec, cdms_sum[0, :], c='k', clip_on=False)
ax_sum_2.plot(tvec, cdms_sum[1, :], c='k', clip_on=False)
ax_sum_3.plot(tvec, cdms_sum[2, :], c='k', clip_on=False)

mark_subplots(ax0, "A", xpos=0.05, ypos=.99)
mark_subplots(ax_ind_1, "B", xpos=-0.05, ypos=1.2)
mark_subplots(ax_sum_1, "C", xpos=-0.05, ypos=1.2)

plt.savefig("pz_remains.pdf")